In [1]:
import pandas as pd
import feather
import numpy as np
import csv
import datetime
pd.set_option('display.max_columns', 30)

In [2]:
data = pd.read_csv("/Volumes/BigData/Science/hse_thesis/metadata_for_every_post_and_merged_texts/merged_data_final.csv", parse_dates=["date"])

/Users/Hun/Python_env/Python3_env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
data.head()

,original_texts,text,author_id,wall_id,post_comment_id_int,city,date,name,region,is_post,surname,sex,country,posts,comments,likes,res_comments,res_postlikes,res_comments_likes,friends,groups,followers,post_likes,post_repost,post_comments,text_length
0,General Lee,general lee,49154141,49154141,2404,Владимир,2012-02-24 11:54:20,Дмитрий,Владимирская область,True,Филиппов,2,Россия,2133.0,0.0,1352.0,0.0,2670.0,0.0,198.0,106.0,113.0,2.0,0.0,0.0,2
1,Shelby GT500,shelby,49154141,49154141,2405,Владимир,2012-02-24 11:54:33,Дмитрий,Владимирская область,True,Филиппов,2,Россия,2133.0,0.0,1352.0,0.0,2670.0,0.0,198.0,106.0,113.0,1.0,0.0,0.0,1
2,1969 Ford Mustang,ford mustang,49154141,49154141,2411,Владимир,2012-02-25 22:27:35,Дмитрий,Владимирская область,True,Филиппов,2,Россия,2133.0,0.0,1352.0,0.0,2670.0,0.0,198.0,106.0,113.0,1.0,0.0,0.0,2
3,Murcielago,murcielago,49154141,49154141,2400,Владимир,2012-02-24 11:53:51,Дмитрий,Владимирская область,True,Филиппов,2,Россия,2133.0,0.0,1352.0,0.0,2670.0,0.0,198.0,106.0,113.0,1.0,0.0,0.0,1
4,Суровый водитель,суровый водитель,49154141,49154141,2392,Владимир,2012-02-23 16:22:56,Дмитрий,Владимирская область,True,Филиппов,2,Россия,2133.0,0.0,1352.0,0.0,2670.0,0.0,198.0,106.0,113.0,1.0,0.0,0.0,2


In [30]:
data.columns

Index(['original_texts', 'text', 'author_id', 'wall_id', 'post_comment_id_int',
       'city', 'date', 'name', 'region', 'is_post', 'surname', 'sex',
       'country', 'posts', 'comments', 'likes', 'res_comments',
       'res_postlikes', 'res_comments_likes', 'friends', 'groups', 'followers',
       'post_likes', 'post_repost', 'post_comments', 'text_length'],
      dtype='object')

In [4]:
data.dtypes

original_texts                 object
text                           object
author_id                       int64
wall_id                         int64
post_comment_id_int             int64
city                           object
date                   datetime64[ns]
name                           object
region                         object
is_post                          bool
surname                        object
sex                            object
country                        object
posts                         float64
comments                      float64
likes                         float64
res_comments                  float64
res_postlikes                 float64
res_comments_likes            float64
friends                       float64
groups                        float64
followers                     float64
post_likes                    float64
post_repost                    object
post_comments                 float64
text_length                     int64
dtype: objec

In [5]:
def aa(val):
    try:
        res = float(val)
        return res
    except ValueError as err:
        print(err, val)
        return np.nan

In [6]:
data["post_repost"] = data["post_repost"].apply(aa)

could not convert string to float:   
could not convert string to float:   


In [7]:
data["original_texts"] = data["original_texts"].apply(lambda x: str(x))
data["text"] = data["text"].apply(lambda x: str(x))

In [8]:
data_good_date = data[data.date >= datetime.date(year=2010,month=10,day=21)]
data_good_date.shape

(5121979, 26)

In [9]:
data_good_date = data_good_date[data_good_date["author_id"] != 0]
data_good_date.shape

(4890025, 26)

In [ ]:
from collections import defaultdict
merged_authors_dict = {}
for author_id, author_data in data_good_date.groupby("author_id"):
    texts_number = len(author_data["text"])
    orig_texts_storage = ""
    texts_storage = ""
    post_likes_storage = 0
    post_repost_storage = 0
    post_comments_storage = 0
    auth_dict = {}
    auth_dict["region"] = author_data["region"].iloc[0]
    auth_dict["city"] = author_data["city"].iloc[0]
    auth_dict["sex"] = author_data["sex"].iloc[0]
    auth_dict["friends"] = author_data["friends"].iloc[0]
    auth_dict["followers"] = author_data["followers"].iloc[0]
    auth_dict["likes"] = author_data["likes"].iloc[0]
    auth_dict["posts"] = author_data["posts"].iloc[0]
    auth_dict["comments"] = author_data["comments"].iloc[0]
    auth_dict["res_comments"] = author_data["res_comments"].iloc[0]
    auth_dict["res_comments_likes"] = author_data["res_comments_likes"].iloc[0]
    auth_dict["res_postlikes"] = author_data["res_postlikes"].iloc[0]
    auth_dict["groups"] = author_data["groups"].iloc[0]
    
    for text_num, text in enumerate(author_data["text"]):
        orig_texts_storage += author_data["original_texts"].iloc[text_num] + " "
        texts_storage += text + " "
        post_likes_storage += author_data["post_likes"].iloc[text_num]
        post_repost_storage += author_data["post_repost"].iloc[text_num]
        post_comments_storage += author_data["post_comments"].iloc[text_num]
        
    auth_dict["original_texts"] = orig_texts_storage
    auth_dict["texts"] = texts_storage
    auth_dict["post_likes"] = post_likes_storage
    auth_dict["post_repost"] = post_repost_storage
    auth_dict["post_comments"] = post_comments_storage
    
    merged_authors_dict[author_id] = auth_dict

In [13]:
len(merged_authors_dict)

379765

In [20]:
next (iter (merged_authors_dict.values()))

{'city': 'Барнаул',
 'comments': 1.0,
 'docs': [],
 'followers': 100.0,
 'friends': 106.0,
 'groups': 43.0,
 'likes': 4.0,
 'original_texts': 'Что пожелать тебе родная,Богатства или красоты?Не лучше ль будет если в жизниВсегда счастливой будешь ты?Чтоб места не было беде,И чтоб никто не догадалсяКоторый год идёт тебе.Бери от жизни всё, что можешь,Всё ,что просто и светло,Ведь жизнь на жизнь не перемножить,А дважды жить не суждено. Что пожелать тебе родная,Богатства или красоты?Не лучше ль будет если в жизниВсегда счастливой будешь ты?Чтоб места не было беде,И чтоб никто не догадалсяКоторый год идёт тебе.Бери от жизни всё, что можешь,Всё ,что просто и светло,Ведь жизнь на жизнь не перемножить,А дважды жить не суждено. ',
 'post_comments': 0.0,
 'post_likes': 0.0,
 'post_repost': 0.0,
 'posts': 1.0,
 'region': 'Алтайский край',
 'res_comments': 0.0,
 'res_comments_likes': 0.0,
 'res_postlikes': 0.0,
 'sex': '1',
 'texts': '{пожелать} {родной} {богатство} {красота} {хороший} {жизнивсегда}

In [19]:
for author in merged_authors_dict:
    merged_authors_dict[author]["texts"] = " ".join(["{" + token + "}"for token in merged_authors_dict[author]["texts"].split()])

In [22]:
with open("/Volumes/BigData/Science/hse_thesis/metadata_for_every_post_and_merged_texts/aggregated_by_author_texts.csv", "wt", newline="") as f:
    fieldnames = ["author_id", "region", "city", "groups", "likes", "sex",
                  "res_comments_likes", "res_comments", "res_postlikes",
                  "friends", "followers", "comments", "text", "o_text",
                  "post_comments", "post_likes", "post_repost", "posts"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for auth_id, val in merged_authors_dict.items():
        row_dict = {
            "author_id": auth_id,
            "region": val["region"],
            "city": val["city"],
            "groups": val["groups"],
            "likes": val["likes"],
            "sex": val["sex"],
            "res_comments": val["res_comments"],
            "res_comments_likes": val["res_comments_likes"],
            "res_postlikes": val["res_postlikes"],
            "friends": val["friends"],
            "followers": val["followers"],
            "comments": val["comments"],
            "text": val["texts"],
            "o_text": val["original_texts"],
            "post_comments": val["post_comments"],
            "post_likes": val["post_likes"],
            "post_repost": val["post_repost"],
            "posts": val["posts"],
        }
        writer.writerow(row_dict)

In [23]:
merged_authors_pd = pd.read_csv("/Volumes/BigData/Science/hse_thesis/metadata_for_every_post_and_merged_texts/aggregated_by_author_texts.csv")

In [26]:
merged_authors_pd.shape

(379765, 18)

In [27]:
fieldnames = ["o_text", "text", "author_id", "region", "city", "groups", "likes", "sex",
              "res_comments_likes", "res_comments", "res_postlikes",
              "friends", "followers", "posts", "comments",
              "post_comments", "post_likes", "post_repost"]
merged_authors_pd = merged_authors_pd[fieldnames]

In [28]:
merged_authors_pd.head()

,o_text,text,author_id,region,city,groups,likes,sex,res_comments_likes,res_comments,res_postlikes,friends,followers,posts,comments,post_comments,post_likes,post_repost
0,"Что пожелать тебе родная,Богатства или красоты...",{пожелать} {родной} {богатство} {красота} {хор...,185597961,Алтайский край,Барнаул,43.0,4.0,1,0.0,0.0,0.0,106.0,100.0,1.0,1.0,0.0,0.0,0.0
1,с днём рождения!),{день} {рождение},144703498,Мордовия,Саранск,86.0,0.0,1,0.0,0.0,0.0,70.0,489.0,1.0,0.0,0.0,0.0,0.0
2,с днем рождения!!!!,{день} {рождение},105906189,Пензенская область,Земетчино,92.0,0.0,2.0,0.0,0.0,0.0,358.0,1630.0,1.0,0.0,0.0,0.0,0.0
3,Мужская солидарность Лемурьей мудрости пост. О...,{мужской} {солидарность} {лемурьей} {мудрость}...,12582933,Севастопольский городской совет,Севастополь,37.0,85.0,2,4.0,15.0,240.0,139.0,47.0,101.0,21.0,14.0,117.0,8.0
4,?Отправлено через https://vk.com/top_cards#386...,{отправлять} {vk} {top} {cards} {vk} {loc} {vk...,139460631,NaN,NaN,66.0,1.0,1.0,0.0,0.0,3.0,105.0,74.0,8.0,0.0,0.0,3.0,0.0


In [29]:
merged_authors_pd.to_csv("/Volumes/BigData/Science/hse_thesis/metadata_for_every_post_and_merged_texts/aggregated_by_author_texts.csv",
                         header=False, index=False, sep=";", quoting=csv.QUOTE_ALL, line_terminator=";\r\n")